# Libraries

In [12]:
# Supress warnings
from typing import Tuple
import warnings; warnings.filterwarnings('ignore')

# Download videos from youtube
from __future__ import unicode_literals
from youtube_dl.utils import DownloadError
import youtube_dl

# Regex
import re

# Image processing
import cv2
import mediapipe as mp

# Progress bar
from tqdm.auto import tqdm

# Webscraping
import bs4
from bs4 import BeautifulSoup as bs

# System
import os

# Custom functions
from functions import process_results, get_video_frames, generate_reference_database

print('OpenCV version: ', cv2.__version__)
print('BeautifulSoup version: ', bs4.__version__)

OpenCV version:  4.5.5
BeautifulSoup version:  4.10.0


## Selenium Configuration

Selenium uses a WebDriver to perform WebScraping. It has built-in support for any browner based on Chromium.  
  
**Instructions:**  

- Download <a href='https://chromedriver.storage.googleapis.com/index.html'>Chrome WebDriver</a>  
- Copy `msedgedriver` file `/usr/local/bin`  
- Open terminal inside `bin` folder and type `xattr -d com.apple.quarantine chromedriver`  
- Upgrade pip `pip3 install --upgrade pip`  
- Install selenium `pip3 install selenium`  
- Verify the installation running the following below  

In [8]:
from selenium import webdriver
import time

print('selenium version: ', webdriver.__version__)

# Add options to the driver object
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Provide the path of msedgedriver in the system
driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=options)
driver.set_window_size(1280,800)

# Send a get request to the url
driver.get('https://www.github.com/baiochi/')
time.sleep(10)
# Close WebDriver
driver.quit()

selenium version:  3.14.1


## Webscraping URLs with `Selenium` and `BeautifulSoup`

Open Youtube page with WebDriver

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
#options.add_argument(' --headless')
driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=options)
driver.set_window_size(720,480)

# Open URL
driver.get('https://www.youtube.com/user/incluirtecnologia/playlists')

Extract page information with *BeautifulSoup*

In [10]:
# Playlist page
soup_file = driver.page_source
soup = bs(soup_file, 'html.parser')

Each playlist has it's URL stored inside **id** tag with value **video-title**

In [11]:
# Extract title/url for each playlist
playlist_urls = {re.sub('\W+',' ', link.get('title')):link.get('href') 
				for link in soup.find_all('a', {'id': 'video-title'})}
playlist_urls

{'Vestuários Objetos Pessoais': '/watch?v=ekKfYA3Al_I&list=PL3NiGvCg31F8wPRAfmnBDapEwM6Vtdci9',
 'Meios de Transporte': '/watch?v=oDuoSoV9zFo&list=PL3NiGvCg31F-mnY-4RR4px40Ao6XcwZO_',
 'Casa': '/watch?v=hwnPfsri8U0&list=PL3NiGvCg31F8D9JsWZNc50RvbN97hhItN',
 'Documentos': '/watch?v=bt7go8DOz1Q&list=PL3NiGvCg31F_D9eVW-BSB_BsDHiHzkUqM',
 'Pronomes': '/watch?v=SPy0Gn7DqF8&list=PL3NiGvCg31F-JVV9VSsQosVQqkaAmWnZg',
 'Lugares': '/watch?v=yZLFEIJkApk&list=PL3NiGvCg31F_C4xMNGW98OPTXWfjX-E73',
 'Natureza': '/watch?v=9wnMp4TCazY&list=PL3NiGvCg31F_qyfr5lNWy93DJcnz30OPC',
 'Cores': '/watch?v=eqDGTazitxA&list=PL3NiGvCg31F8OQ5OSP4I7pXmg11fjQr5k',
 'Alimentos': '/watch?v=Y41tSCSZzoI&list=PL3NiGvCg31F_pEw1KzN-y1kPMXvworJdE',
 'Identidade Cumprimentos': '/watch?v=_X2i1MXPCkA&list=PL3NiGvCg31F-WZYf2UJX6yBmpq8uLXjMH',
 'Pessoas Família': '/watch?v=T3JYpB_gF2U&list=PL3NiGvCg31F_ZWackAuL0fejRL61YYJko',
 'Localidades': '/watch?v=DS7RDmwJqdE&list=PL3NiGvCg31F-x5c80oDyI6Pdsz7sCIVq9',
 'Profissões': '/watch?v=w

Close WebDriver.

In [12]:
driver.quit()

## Downloading files with `youtube_dl`

Create folders for each playlist

In [2]:
!ls -la ..

total 2528
drwxr-xr-x@ 11 baiochi  staff      352 May  9 14:49 .
drwxr-xr-x@  7 baiochi  staff      224 May  3 12:24 ..
-rw-------@  1 baiochi  staff    14340 May  9 16:27 .DS_Store
drwxr-xr-x@  4 baiochi  staff      128 May  4 19:04 .ipynb_checkpoints
-rw-------@  1 baiochi  staff  1277380 May  7 11:22 Sign Language Detection.ipynb
drwxr-xr-x@  3 baiochi  staff       96 May  7 12:39 __pycache__
drwxr-xr-x@  9 baiochi  staff      288 May  6 11:14 img
drwxr-xr-x@  6 baiochi  staff      192 May  9 14:48 src
drwxr-xr-x@  4 baiochi  staff      128 May  6 00:52 styles
drwxr-xr-x@  4 baiochi  staff      128 May  6 22:16 trash
drwxr-xr-x@ 29 baiochi  staff      928 May  9 14:43 video


In [35]:
for folder in playlist_urls.keys():
	os.mkdir(f'../database/{folder}')

Download all videos from playlists

In [45]:
for folder, url in playlist_urls.items():

    # Download in subfolder 'video/[playlist_name]', file name format example: 01-Libras -A.mp4
    ydl_opts = {
        'outtmpl'  : f'{os.path.abspath("..")}/database/{folder}' + u'/%(playlist_index)s-%(title)s.%(ext)s',
        'cachedir' : False
        }
    # Complete url
    url = 'https://www.youtube.com' + url
    # Create YoutubeDL object
    ydl = youtube_dl.YoutubeDL(ydl_opts)
    # Download videos from playlist
    try:
        ydl.download([url])
    except DownloadError:
        print(u'Unable to download from folder {foler}')

[youtube:tab] Downloading playlist PL3NiGvCg31F8wPRAfmnBDapEwM6Vtdci9 - add --no-playlist to just download video ekKfYA3Al_I
[youtube:tab] PL3NiGvCg31F8wPRAfmnBDapEwM6Vtdci9: Downloading webpage
[youtube:tab] PL3NiGvCg31F8wPRAfmnBDapEwM6Vtdci9: Downloading webpage
[download] Downloading playlist: Vestuários / Objetos Pessoais
[youtube:tab] playlist Vestuários / Objetos Pessoais: Downloading 37 videos
[download] Downloading video 1 of 37
[youtube] ekKfYA3Al_I: Downloading webpage
[download] /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Vestuários Objetos Pessoais/01-Libras - Pente.mp4 has already been downloaded
[download] 100% of 130.71KiB
[download] Downloading video 2 of 37
[youtube] iJs9G_QD3Kc: Downloading webpage
[download] /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Vestuários Objetos Pessoais/02-Libras - Chapéu.mp4 has already been downloaded
[download] 100% of 150.00KiB
[download] Downloading video 3 of 37
[youtube] rFzw

ERROR: unable to download video data: HTTP Error 403: Forbidden


Unable to download file %(title)s.%(ext)s
[youtube:tab] Downloading playlist PL3NiGvCg31F_D9eVW-BSB_BsDHiHzkUqM - add --no-playlist to just download video bt7go8DOz1Q
[youtube:tab] PL3NiGvCg31F_D9eVW-BSB_BsDHiHzkUqM: Downloading webpage
[youtube:tab] PL3NiGvCg31F_D9eVW-BSB_BsDHiHzkUqM: Downloading webpage
[download] Downloading playlist: Documentos
[youtube:tab] playlist Documentos: Downloading 22 videos
[download] Downloading video 1 of 22
[youtube] bt7go8DOz1Q: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Documentos/01-Libras - IPTU.mp4
[download] 100% of 278.43KiB in 00:0671KiB/s ETA 00:000
[download] Downloading video 2 of 22
[youtube] LF87pKIX4lE: Downloading webpage
[youtube] LF87pKIX4lE: Downloading MPD manifest


ERROR: unable to download video data: HTTP Error 403: Forbidden


Unable to download file %(title)s.%(ext)s
[youtube:tab] Downloading playlist PL3NiGvCg31F-JVV9VSsQosVQqkaAmWnZg - add --no-playlist to just download video SPy0Gn7DqF8
[youtube:tab] PL3NiGvCg31F-JVV9VSsQosVQqkaAmWnZg: Downloading webpage
[youtube:tab] PL3NiGvCg31F-JVV9VSsQosVQqkaAmWnZg: Downloading webpage
[download] Downloading playlist: Pronomes
[youtube:tab] playlist Pronomes: Downloading 5 videos
[download] Downloading video 1 of 5
[youtube] SPy0Gn7DqF8: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Pronomes/1-Libras - Seu.mp4
[download] 100% of 465.83KiB in 00:1152KiB/s ETA 00:000
[download] Downloading video 2 of 5
[youtube] 1nDm4BFSepA: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Pronomes/2-Libras - Nós.mp4
[download] 100% of 538.33KiB in 00:0943KiB/s ETA 00:003
[download] Downloading video 3 of 5
[youtube] BEOe6bFd2H0: Downloading webp

...

Unable to download file %(title)s.%(ext)s
[youtube:tab] Downloading playlist PL3NiGvCg31F9DG-1zu-rSlA-Vg9IhxftG - add --no-playlist to just download video 1IAeXkUHVFg
[youtube:tab] PL3NiGvCg31F9DG-1zu-rSlA-Vg9IhxftG: Downloading webpage
[youtube:tab] PL3NiGvCg31F9DG-1zu-rSlA-Vg9IhxftG: Downloading webpage
[download] Downloading playlist: Números
[youtube:tab] playlist Números: Downloading 20 videos
[download] Downloading video 1 of 20
[youtube] 1IAeXkUHVFg: Downloading webpage
[youtube] 1IAeXkUHVFg: Downloading MPD manifest
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Números/01-Libras - 0.mp4
[download] 100% of 438.28KiB in 00:0781KiB/s ETA 00:000
[download] Downloading video 2 of 20
[youtube] L33J7IQWQlg: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Números/02-Libras - 1.mp4
[download] 100% of 661.91KiB in 00:1568KiB/s ETA 00:001
[download] Downloading video 3

ERROR: unable to download video data: HTTP Error 403: Forbidden


Unable to download file %(title)s.%(ext)s
[youtube:tab] Downloading playlist PL728A4516E8628D08 - add --no-playlist to just download video UvWb6tBUx6w
[youtube:tab] PL728A4516E8628D08: Downloading webpage
[youtube:tab] PL728A4516E8628D08: Downloading webpage
[download] Downloading playlist: Glossário de Libras
[youtube:tab] Downloading page 1
[youtube:tab] Downloading page 2
[youtube:tab] playlist Glossário de Libras: Downloading 231 videos
[download] Downloading video 1 of 231
[youtube] UvWb6tBUx6w: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Glossário de Libras/001-Libras - Tudo bem.mp4
[download] 100% of 114.93KiB in 00:0256KiB/s ETA 00:00
[download] Downloading video 2 of 231
[youtube] UvWb6tBUx6w: Downloading webpage
[download] Destination: /Users/baiochi/OneDrive/GitHub/LetsCode/7_Unstructured_Data/Project/video/Glossário de Libras/002-Libras - Tudo bem.mp4
[download] 100% of 114.93KiB in 00:0201KiB/s ETA 0

ERROR: unable to download video data: HTTP Error 403: Forbidden


Unable to download file %(title)s.%(ext)s


## Create Sign Database

In [3]:
# Hands object
hands = mp.solutions.hands.Hands(model_complexity=0, 
								min_detection_confidence=0.5, 
								min_tracking_confidence=0.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
reference_sign = {}

In [15]:
for folder in sorted(os.listdir('../database'))[2:]:

	reference_sign[folder] = generate_reference_database(
								sorted(os.listdir(f'../database/{folder}')), 
								path=os.path.abspath(f'../database/{folder}'), 
								model=hands)

Reading frames: 100%|██████████| 98/98.0 [00:00<00:00, 106.00it/s]
Calculating left landmark angles: 100%|██████████| 54/54 [00:21<00:00,  2.49it/s]
Reading frames: 100%|██████████| 92/92.0 [00:00<00:00, 103.98it/s]
Calculating left landmark angles: 100%|██████████| 46/46 [00:18<00:00,  2.43it/s]
Reading frames: 100%|██████████| 87/87.0 [00:00<00:00, 100.11it/s]
Calculating right landmark angles: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]
Reading frames: 100%|██████████| 104/104.0 [00:00<00:00, 110.01it/s]
Calculating right landmark angles: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]
Reading frames: 100%|██████████| 107/107.0 [00:01<00:00, 103.87it/s]
Calculating left landmark angles: 100%|██████████| 50/50 [00:20<00:00,  2.38it/s]
Reading frames: 100%|██████████| 94/94.0 [00:00<00:00, 111.63it/s]
Calculating left landmark angles: 100%|██████████| 49/49 [00:26<00:00,  1.84it/s]
Reading frames: 100%|██████████| 95/95.0 [00:00<00:00, 126.73it/s]
Calculating left landmark angles: 100

UnboundLocalError: local variable 'frame_list' referenced before assignment

## Save object with `Pickle`

In [21]:
import pickle

with open('../database/reference_signs.pickle', 'wb') as file:
    pickle.dump(reference_sign, file, protocol=pickle.HIGHEST_PROTOCOL)